# Extracción de variables operativas de la CNBV

In [36]:
##    Las variables operativas se refieren a las características de diferentes instituciones 
##    financieras tales como: el número de sucursales, tdc, tpv, cajeros. Estas se encuentran 
##    a nivel mes-municipio.


In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from random import seed
from random import randint
import pandas as pd
import re 
import shutil
import os

### Hardcoded: path para guardar los archivos descargados

In [38]:
path_download = "/home/ant/Downloads/"
path_save = "/home/ant/Documents/Tesis/cnbv_data/"

## Extraer los checkboxes

In [39]:
url = 'https://picomponentebi.cnbv.gob.mx/ReportViwer/ReportService?sector=40&tema=124&subTema=0&tipoInformacion=0&subTipoInformacion=0&idReporte=40S2422&idPortafolio=0&idTipoReporteBI=1'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(10)
ids = driver.find_elements_by_xpath('//*[@id]')
lista_ids = []
for ii in ids:
    lista_ids.append(ii.get_attribute('id')) 

In [40]:
subs = '_\d\d\d\d\d\d'
fechas = [x for x in lista_ids if re.search(subs, x)] 

In [41]:
subs = 'chk_[a-zA-Z]'
bancos = [x for x in lista_ids if re.search(subs, x)] 
bancos = bancos[2:]

In [49]:
def get_data(fecha,banco):
    
    print("Fecha: ",fecha)
    print("Banco: ",banco)
    
    timmer = randint(10,20)
    print('Inicia timer de...{} segundos'.format(timmer))
    time.sleep(timmer)
    
    # Ir a la página
    url = 'https://picomponentebi.cnbv.gob.mx/ReportViwer/ReportService?sector=40&tema=124&subTema=0&tipoInformacion=0&subTipoInformacion=0&idReporte=40S2422&idPortafolio=0&idTipoReporteBI=1'
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(10)
    
    # Filtros
    elem1 = driver.find_elements_by_xpath("//button[@id='btn_OcultaFiltros']")
    time.sleep(5)
    elem1[0].click()
    time.sleep(5)
    
    #Customize filtros
    driver.find_element_by_css_selector("input[type='radio'][value='14']").click()
    
    print("--------- FECHAS ---------")
    #este es el default 
    if (fecha != 'chk_201912'):
        # Seleccionar una fecha
        checkboxElement = driver.find_element_by_id("{}".format(fecha))
        checkboxElement.click()
        time.sleep(5)
        
         # De-seleccionar el default
        checkboxElement = driver.find_element_by_id("chk_201912")
        checkboxElement.click()
        time.sleep(5)
    #Cuando sí sea, ya está seleccionado y seguimossss
    
    #Institución
    driver.find_element_by_id("Institucion_6_H").click()
    time.sleep(5)
    
    print("--------- INSTITUCION ---------")
    if (banco != 'chk_ABC_Capital'):
        # Seleccionar institución
        check_box = driver.find_element_by_id("{}".format(banco))
        check_box.click()
        time.sleep(5)
        
        #Deseleccionar
        checkboxElement = driver.find_element_by_id("chk_ABC_Capital")
        checkboxElement.click()
        time.sleep(5)
        
    
    #Generar reporte
    element = driver.find_elements_by_xpath("//button[@id='btn_GeneraRoporte']")
    element[0].click()
    time.sleep(15)
    
    ####Descargar datos
    # cambiar de frame
    driver.switch_to.frame('IFrame_Container')
    time.sleep(5)

    #cambiar al 2do frame
    frame3 = driver.find_element_by_xpath("//iframe[@src='/ReportViewerWebForm.aspx']")
    driver.switch_to.frame(frame3)
    time.sleep(5)

    #drop down 
    checkbox = driver.find_element_by_xpath("//img[@id='ReportViewer1_ctl05_ctl04_ctl00_ButtonImgDown']")
    checkbox.click()
    time.sleep(5)

    #descargar archivo
    checkbox2 = driver.find_element_by_xpath("//a[@title='CSV (delimitado por comas)']")
    checkbox2.click()
    
    timmer = randint(15,30)
    print('Inicia timer de...{} segundos'.format(timmer))
    time.sleep(timmer)
    
    new_name = fecha.replace("chk_","") + "_" + banco.replace("chk_","") + ".csv" 
    #current_name = path_download + 'S2422.csv' 
    #current_path_new_name = path_download + new_name
    
    os.rename(path_download + 'S2422.csv',path_download + new_name)
    
    #download = path_download +  '{}'.format(new_name)
    #save = path_save + '{}'.format(new_name)
    shutil.move(path_download +  new_name, path_save + new_name)

    print("---- ARCHIVO CORRECTAMENTE DESCARGADO ----")


# Descargar data

In [53]:
for i in fechas:
    for j in bancos[38:]:
        get_data(i,j)

In [3]:
# Desde aquí : Fecha:  chk_201912
# Banco:  chk_Banorte/Ixe

In [52]:
bancos[38:]

['chk_Banco_Bancrea',
 'chk_Banorte/Ixe',
 'chk_Inbursa',
 'chk_Scotiabank',
 'chk_Banco_Base',
 'chk_Banregio',
 'chk_ING',
 'chk_The_Bank_of_New_York_Mellon',
 'chk_Banco_Bicentenario',
 'chk_Bansí',
 'chk_Inmobiliario_Mexicano',
 'chk_UBS']